## Imports

In [ ]:
%load_ext lab_black

import h5py
import os
import numpy as np
from typing import Dict, List, Optional, Tuple

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm


import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt


import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    kw_data_loader,
    cnn,
)

colors = pc.qualitative.D3
colors_seq = pc.sequential.dense
angstrom = "\u212B"
Theta = "\u0398"

In [ ]:
def average_imgs(ddir, files, new_filename=None, xlim=None, ylim=None):
    all_vals = []
    for file in files:
        data, theta, energy = loading_functions.load_hdf5(ddir, file)
        all_vals.append(
            analysis_functions.limit_dataset(theta, energy, data, xlim=xlim, ylim=ylim)
        )

    x_vals, y_vals, d_vals = zip(*all_vals)
    data_avg = np.mean(d_vals, axis=0)
    return x_vals[0], y_vals[0], data_avg


def sum_imgs(ddir, files, new_filename=None, xlim=None, ylim=None):
    all_vals = []
    for file in files:
        data, theta, energy = loading_functions.load_hdf5(ddir, file)
        all_vals.append(
            analysis_functions.limit_dataset(theta, energy, data, xlim=xlim, ylim=ylim)
        )

    x_vals, y_vals, d_vals = zip(*all_vals)
    data_avg = np.sum(d_vals, axis=0)
    return x_vals[0], y_vals[0], data_avg

In [ ]:
def generate_streak_corrected_dataset(
    ad,
    fp,
    fn,
    correction_array,
    xlim=None,
    ylim=None,
    x_bin=1,
    y_bin=1,
    int_range=0.0,
    overwrite=False,
    new_fn=None,
):
    new_fn = (
        new_fn
        if new_fn
        else f"{os.path.splitext(fn)[0]}_streakCorrect_{int_range:.2f}int.h5"
    )
    new_fn = os.path.join(fp, new_fn)
    if not overwrite and os.path.exists(new_fn):
        raise FileExistsError(f"{new_fn} already exists")
    new_data = []
    for p in ad.phi_or_time:
        val = p
        xaxis, yaxis, dataslice = tr_functions.slice_datacube(
            ad,
            slice_dim="z",
            slice_val=val,
            int_range=int_range,
            xlim=xlim,
            ylim=ylim,
            x_bin=x_bin,
            y_bin=y_bin,
            norm_data=False,
            plot_data=False,
        )
        corrected_data = correction_array * dataslice
        new_data.append(corrected_data)
    new_data = np.array(new_data).T
    with h5py.File(
        new_fn, "w"
    ) as f:  # Note: 'w' creates a new empty file (or overwrites), use 'r+' to modify an existing file
        f["data"] = new_data
        axes_names = [
            "theta",
            "energy",
            "phi",
        ]  # Change these to match your axes labels
        axes = [xaxis, yaxis, ad.phi_or_time]
        for axis, name in zip(axes, axes_names):
            f[name] = axis
    return new_fn

# Load Data

In [ ]:
# ## Convert lamp data ##

# ddir = r"E:\atully\arpes_data\2023_February\Lamp"

# # STEP 1 ##
# # Convert ibw to hdf5
# fn = "Img7_Lamp_g_kw.ibw"
# HDF5_loader.ibw_to_hdf5(ddir, fn, export=True)

# # Check conversion worked
# data, theta, energy = HDF5_loader.load_hdf5(
#     ddir, "Img7_Lamp_g_kw.h5"
# )  # load data from hdf5
# data.shape, theta.shape, energy.shape

In [ ]:
colors = pc.qualitative.D3

# Fluence Dependence

In [ ]:
ddir = r"E:\atully\arpes_data\2023_June\C60\ARPES\Imgs\Fluence Dependence"

## Ecenter 2.15, aquisitions 76, zero delay, TR0

files = []
files = [f"Img_00{i}.h5" for i in range(1, 3)]
files.extend(f"Img_00{i}.h5" for i in range(0, 1))
files.extend(f"Img_00{i}.h5" for i in range(3, 9))


names = [
    "neg delay",
    "pump blocked",
    "2 mW",
    "2.5 mW",
    "3 mW",
    "3.5 mW",
    "4 mW",
    "4.5 mW",
]

x_power = np.array([-1, 0, 2, 2.5, 3, 3.5, 4, 4.5])

colors = pc.qualitative.D3
# data, theta, energy = loading_functions.load_hdf5(ddir, file)

In [ ]:
## Limit Dataset ##

xlim = None
# ylim = None
ylim = (energy[57], energy[1007])  # get rid of zero padding on datasets

all_vals = []
for file in files:
    data, theta, energy = loading_functions.load_hdf5(ddir, file)
    all_vals.append(analysis_functions.limit_dataset(theta, energy, data, xlim, ylim))

In [ ]:
## Plot Data ##

x_1, y_1, d_1 = all_vals[0]

fig = tr_functions.thesis_fig(
    title=f"{file}",
    xaxis_title="$theta$",
    yaxis_title="Energy (eV)",
    equiv_axes=False,
)

fig.add_trace(
    go.Heatmap(x=x_1, y=y_1, z=analysis_functions.norm_data(d_1), coloraxis="coloraxis")
)

fig.show(renderer="svg")

In [ ]:
## Get and Plot 1D Data ##

fig = tr_functions.thesis_fig(
    title=f"EDCs: Fluence Dependence",
    xaxis_title="Energy (eV)",
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=1000,
)

total_counts = []

for i in range(len(files) - 1):
    x, y, d = all_vals[i]
    y, col = tr_functions.get_1d_x_slice(x=x, y=y, data=d, ylims=None, x_range=None)

    # Plot Data
    color = colors[i % len(colors)]
    fig.add_trace(go.Scatter(x=y, y=col, name=names[i], line=dict(color=color)))

    total_counts.append(np.sum(col))

fig.show()

# fig.write_image(r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\prelim_EDCs.png")

In [ ]:
fig = tr_functions.thesis_fig(
    title=f"Fluence Dependence Linearity Check",
    xaxis_title="Power (mW)",
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=1000,
)

fig.add_trace(go.Scatter(x=x_power, y=total_counts, line=dict(color=colors[0])))

# HS Cuts

In [ ]:
xaxis_title = f"{Theta}"
yaxis_title = f"E<sub>K</sub> (eV)"

Starting with img_011, accumulate was ticked on, so can only take last image. Don't average cycles together.

In [ ]:
theta, energy, data = average_imgs(ddir, files)

In [ ]:
# HS Cuts -- sums

ddir = r"E:\atully\arpes_data\2023_June\C60\ARPES\Imgs\HS"

files = []

In [ ]:
## G @ phi=-1 ##

title = "G1 (phi=-1)"
g_offset = -2
files = []

# Sum files
files = [f"Img_002_{i}.h5" for i in range(1, 6)]
files.extend(f"Img_003_{i}.h5" for i in range(1, 11))
files.extend(f"Img_022_{i}.h5" for i in range(6, 7))  # take last img only
theta_avg, energy_avg, data_avg = sum_imgs(ddir, files)
g_theta, g_energy, g_data = sum_imgs(ddir, files)

In [ ]:
## G @ phi=3.5 ##

title = "G2 (phi=3.5)"
g_offset = -2

# Sum files
files = []
files = [f"Img_009_{i}.h5" for i in range(1, 11)]  # okay to avg
files.extend(f"Img_010_{i}.h5" for i in range(1, 11))  # okay to avg
files.extend(f"Img11.h5" for i in range(1, 2))
files.extend(f"Img_011_{i}.h5" for i in range(10, 11))
files.extend(f"Img_021_{i}.h5" for i in range(10, 11))
theta_avg, energy_avg, data_avg = sum_imgs(ddir, files)
# # theta_avg, energy_avg, data_avg = average_imgs(ddir, files)

# # Last images only
# files = []
# files = [f"Img_011_{i}.h5" for i in range(10, 11)]
# files.extend(f"Img_021_{i}.h5" for i in range(10, 11))
# theta2, energy2, data2 = average_imgs(ddir, files)
# # theta_avg, energy_avg, data_avg = average_imgs(ddir, files)

# theta_avg, energy_avg, data_avg = theta2, energy2, np.mean([data1, data2], axis=0)

# # files.extend(f"Img11.h5" for i in range(1, 2))  # single img -- don't include

In [ ]:
## K1 -- left side of BZ ##

title = "K1"
offset = 17.7

# Sum files
files = []
files = [f"Img_004_{i}.h5" for i in range(1, 11)]
files.extend(f"Img_030_{i}.h5" for i in range(1, 11))
theta_avg, energy_avg, data_avg = sum_imgs(ddir, files)
k1_theta, k1_energy, k1_data = sum_imgs(ddir, files)

In [ ]:
## K2 -- right side of BZ ##

title = "K2"
offset = -22

# Sum files
files = []
files = [f"Img_005_{i}.h5" for i in range(1, 11)]
theta_avg, energy_avg, data_avg = sum_imgs(ddir, files)
k2_theta, k2_energy, k2_data = sum_imgs(ddir, files)

In [ ]:
## M3 -- left side of BZ ##

title = "M3"
offset = 8

# Sum files
files = []
files = [f"Img_006_{i}.h5" for i in range(1, 11)]
files.extend(f"Img_019_{i}.h5" for i in range(10, 11))
theta_avg, energy_avg, data_avg = sum_imgs(ddir, files)
m3_theta, m3_energy, m3_data = sum_imgs(ddir, files)

In [ ]:
## M2 -- right side of BZ ##

title = "M2"
offset = -12

# Sum files
files = []
files = [f"Img_007_{i}.h5" for i in range(1, 11)]
files.extend(f"Img_020_{i}.h5" for i in range(10, 11))
theta_avg, energy_avg, data_avg = sum_imgs(ddir, files)
m2_theta, m2_energy, m2_data = sum_imgs(ddir, files)

In [ ]:
## M1 -- bottom of BZ ##

title = "M1"

# OK to avg
files = []
files = [f"Img_008_{i}.h5" for i in range(1, 11)]
files.extend(f"Img_015_{i}.h5" for i in range(10, 11))
files.extend(f"Img_016_{i}.h5" for i in range(1, 2))
files.extend(f"Img_017_{i}.h5" for i in range(10, 11))
theta_avg, energy_avg, data_avg = sum_imgs(ddir, files)
m1_theta, m1_energy, m1_data = sum_imgs(ddir, files)

In [ ]:
## M1 -- bottom of BZ, neg delay ##

title = "M1 (bottom of BZ; -2 ps)"

# Last images only
files = []
files = [f"Img_018_{i}.h5" for i in range(5, 6)]
theta_avg, energy_avg, data_avg = average_imgs(ddir, files)

In [ ]:
## Cone check (G, pump blocked) ##

title = "Cone check (G, pump blocked)"

# OK to avg
files = []
# files = [f"Img_023_{i}.h5" for i in range(10, 11)]
files = [f"Img29.h5" for i in range(10, 11)]
# files = [f"Img_033_{i}.h5" for i in range(1, 9)]  # cone visible
theta_avg, energy_avg, data_avg = average_imgs(ddir, files)

In [ ]:
## CT2: G @ phi=3.5 ##

title = "CT2: G (phi=3.5)"

# Last images only
files = []

files = [f"Img_012_{i}.h5" for i in range(10, 11)]

# files = [f"Img_014_{i}.h5" for i in range(5, 6)]
# files.extend(f"Img_012_{i}.h5" for i in range(10, 11))
# files.extend(f"Img_013_{i}.h5" for i in range(10, 11))
theta_avg, energy_avg, data_avg = sum_imgs(ddir, files)

In [ ]:
## CT2: G @ phi=-1 ##

title = "CT2: G (phi=-1)"

# Last images only
files = []

files = [f"Img_031_{i}.h5" for i in range(1, 11)]
files.extend(f"Img_032_{i}.h5" for i in range(1, 11))

theta_avg, energy_avg, data_avg = sum_imgs(ddir, files)
ct2_theta, ct2_energy, ct2_data = sum_imgs(ddir, files)

In [ ]:
# ## Subtract neg  c delay (multiphoton)  ##

# data_neg, theta_neg, energy_neg = loading_functions.load_hdf5(ddir, "Img_000.h5")
# data_sub = data - data_neg

In [ ]:
## Limit Dataset and FFT ##

theta, energy, data_new = theta_avg, energy_avg, data_avg
# data_new = data
# data_new = data_sub

xlim = None
ylim = None
# ylim = (energy[57], energy[1007])  # get rid of zero padding on datasets
ylim = (energy[57], 2.45)  # no padding and no curve of detector
xlim = (-17, 16)

## G
# xlim = (-17.4, 15.6)

## K1
# xlim = (-14, 16)

## K2
# xlim = (-19, 15)

## M3
# xlim = (-17, 16)

## M2
# xlim = (-18, 16)

## M1
# xlim = (-18, 16)

## G -- phi = +3.5
# xlim = (-17, 16)

## CT2 -- at G, phi=3.5
# xlim = (-19, 16.7)
# ylim = (2.34, 3.0)


f_data = filter_functions.fft2d_mask(data_new, plot=False)

x_avg, y_avg, d_avg = analysis_functions.limit_dataset(
    theta,
    energy,
    f_data,
    xlim=xlim,
    ylim=ylim,
)

# d_avg = analysis_functions.norm_data(d_avg)

In [ ]:
## Plot ##

x_plot, y_plot, data_plot = (
    x_avg,
    # x_avg - offset + g_offset,
    y_avg,
    d_avg,
)  # averaged or summmed image


fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    height=600,
    width=600,
)

fig.add_trace(
    go.Heatmap(
        x=x_plot,  # fix incorrect traces by, first, plotting without x-axis
        y=y_plot,  # fix incorrect traces by, first, plotting without y-axis
        # z=data_plot,
        z=analysis_functions.norm_data(data_plot),
        # z=np.log(data_plot),
        coloraxis="coloraxis",
    )
)

fig.update_coloraxes(cmin=0, cmax=0.3)
# fig.update_coloraxes(cmin=0.05, cmax=0.15)
# fig.update_coloraxes(cmin=0.05, cmax=0.18)
# fig.update_coloraxes(cmin=0.4, cmax=1.8)  # logscale

fig.show()

In [ ]:
## Get and Plot 1D Data --> EDC ##
xlim = (3, 7)
ylim = None

fig = tr_functions.thesis_fig(
    title=f"EDC",
    xaxis_title="Intensity (arb. u)",
    yaxis_title=yaxis_title,
    equiv_axes=False,
    gridlines=False,
    height=1000,
    width=600,
    # height=800,
    # width=500,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=x_plot, y=y_plot, data=data_plot, ylims=ylim, x_range=xlim
)

# fig.add_trace(go.Scatter(x=y_1d, y=col, line=dict(color=colors[0])))
fig.add_trace(go.Scatter(x=col, y=y_1d, line=dict(color=colors[0])))

fig.show()

## Get and Plot 1D Data --> MDC ##
xlim = None
ylim = (2.15, 2.25)
ylim = None

fig = tr_functions.thesis_fig(
    title=f"MDC",
    xaxis_title=xaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    # height=600,
    # width=800,
)

x_1d, row = tr_functions.get_1d_y_slice(
    x=x_plot, y=y_plot, data=data_plot, xlims=xlim, y_range=ylim
)

fig.add_trace(go.Scatter(x=x_1d, y=row, line=dict(color=colors[0])))

fig.show()

# Stitch Datasets

In [ ]:
title = "CT1 & CT2 at G"

In [ ]:
theta1, energy1, data1 = g_theta, g_energy, g_data  # CT1
theta2, energy2, data2 = ct2_theta, ct2_energy, ct2_data  # CT2

In [ ]:
f_data1 = filter_functions.fft2d_mask(data1, plot=False)
f_data2 = filter_functions.fft2d_mask(data2, plot=False)

In [ ]:
## Stitch and Average Datasets ##

xlim = None
ylim = None

xlim = (-18, 16)
ylim1 = (energy1[57], 2.4)
ylim2 = (energy2[57], 3.0)

x1, y1, dataslice1 = analysis_functions.limit_dataset(
    theta1,
    energy1,
    f_data1,
    xlim=xlim,
    ylim=ylim1,
)

x2, y2, dataslice2 = analysis_functions.limit_dataset(
    theta2,
    energy2,
    f_data2,
    xlim=xlim,
    ylim=ylim2,
)

xs, ys, ds = tr_functions.stitch_and_avg(
    x1,
    y1,
    dataslice1,
    x2,
    y2,
    dataslice2,
    no_avg=True,
)

In [ ]:
## Plot Data ##
x, y, data = xs, ys, ds

fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    height=600,
    width=600,
)

fig.add_trace(
    go.Heatmap(x=x, y=y, z=analysis_functions.norm_data(data), coloraxis="coloraxis")
)

fig.update_coloraxes(cmin=0.07, cmax=0.18)

fig.show()

# Lamp Data

In [ ]:
title = f"C<sub>60</sub> Band Structure"
xaxis_title = f"{Theta}"
yaxis_title = f"E<sub>K</sub> (eV)"

In [ ]:
ddir = r"E:\atully\arpes_data\2023_June\C60\ARPES\Lamp\Swept"

# file = "Img7_Lamp_g_kw.h5"

files = []

## Swept Mode: Measurement Side
# files = [f"Img0.h5" for i in range(1, 2)]
# files.extend(f"Img_001_{i}.h5" for i in range(1, 13))
# theta_img0, energy_img0, data_img0 = sum_imgs(ddir, files)

## Swept Mode: Bad LEED Sid
# files = [f"Img2.h5" for i in range(1, 2)]
# theta_img2, energy_img2, data_img2 = sum_imgs(ddir, files)

theta_avg, energy_avg, data_avg = sum_imgs(ddir, files)

EF = 16.869

In [ ]:
## Limit Dataset and FFT ##

theta, energy, data_new = theta_avg, energy_avg, data_avg
# theta, energy, data_new = theta_avg, energy_avg, data_img0 - data_img2

xlim = None
ylim = None
# ylim = (energy[57], energy[1007])  # get rid of zero padding on datasets
# ylim = (energy[57], 2.45)  # no padding and no curve of detector
xlim = (-20, 17.45)

f_data = filter_functions.fft2d_mask(data_new, plot=False)

x_avg, y_avg, d_avg = analysis_functions.limit_dataset(
    theta,
    energy,
    f_data,
    xlim=xlim,
    ylim=ylim,
)

In [ ]:
## Plot ##

x_plot, y_plot, data_plot = (
    x_avg,
    y_avg,
    d_avg,
)


fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    height=600,
    width=600,
)

fig.add_trace(
    go.Heatmap(
        x=x_plot,
        y=y_plot,
        z=analysis_functions.norm_data(data_plot),
        coloraxis="coloraxis",
    )
)

# fig.update_coloraxes(cmin=0, cmax=1)
fig.update_coloraxes(colorscale="RdBu", cmid=0, showscale=True)

fig.show()

In [ ]:
## Get and Plot 1D Data ##

fig = tr_functions.thesis_fig(
    title=f"EDC of Lamp Data",
    yaxis_title="Energy (eV)",
    xaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=700,
    width=400,
    dtick_y=1,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=x_plot, y=y_plot, data=data_plot, ylims=None, x_range=None
)

# Plot Data
fig.add_trace(go.Scatter(x=col, y=y_1d, name="data", line=dict(color=colors[0])))

fig.show()

# Fixed Mode

In [ ]:
ddir = r"E:\atully\arpes_data\2023_June\C60\ARPES\Lamp\Fixed"

files = []

## Fixed Mode: Measurement Side
files = [f"Img0.h5" for i in range(1, 2)]
theta_img0, energy_img0, data_img0 = sum_imgs(ddir, files)

## Fixed Mode: Bad LEED Side
# files = [f"Img2.h5" for i in range(1, 2)]
# theta_img2, energy_img2, data_img2 = sum_imgs(ddir, files)

theta_avg, energy_avg, data_avg = sum_imgs(ddir, files)

EF = 16.869

In [ ]:
## Limit Dataset and FFT ##

theta, energy, data_new = theta_avg, energy_avg, data_avg
# theta, energy, data_new = theta_avg, energy_avg, data_img0 - data_img2

xlim = None
ylim = None
# ylim = (energy[57], energy[1007])  # get rid of zero padding on datasets
# ylim = (energy[57], 2.45)  # no padding and no curve of detector
xlim = (-20, 17.45)
ylim = (14.14, 15.47)

f_data = filter_functions.fft2d_mask(data_new, plot=False)

x_avg, y_avg, d_avg = analysis_functions.limit_dataset(
    theta,
    energy,
    f_data,
    xlim=xlim,
    ylim=ylim,
)

In [ ]:
## Plot ##

x_plot, y_plot, data_plot = (
    x_avg,
    y_avg,
    d_avg,
)


fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
)

fig.add_trace(
    go.Heatmap(
        x=x_plot,
        y=y_plot,
        z=analysis_functions.norm_data(data_plot),
        coloraxis="coloraxis",
    )
)

fig.update_coloraxes(cmin=0, cmax=0.15)

fig.show()

In [ ]:
## Get and Plot 1D Data ##

fig = tr_functions.thesis_fig(
    title=f"EDC of Lamp Data",
    xaxis_title="Energy (eV)",
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=x_plot, y=y_plot, data=data_plot, ylims=None, x_range=None
)

# Plot Data
fig.add_trace(go.Scatter(x=y_1d, y=col, name="data", line=dict(color=colors[0])))

fig.show()

In [ ]:
## Fit Data ##

x = y_1d
data = analysis_functions.norm_data(col)

offset_type = "constant"

## Offset
c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

## Gaussian
gauss = fitting_functions.make_gaussian(num="A_", amplitude=1, center=14.8, sigma=0.5)

## Lorentzians
# lorentz1 = fitting_functions.make_lorentzian(
#     num="A_", amplitude=0.1, center=14.8, sigma=0.2
# )
# lorentz2 = fitting_functions.make_lorentzian(
#     num="B_", amplitude=0.1, center=15.1, sigma=0.5
# )
lorentz1 = fitting_functions.make_lorentzian(
    num="A_", amplitude=0.1, center=-2, sigma=0.2
)

# full_model = lorentz1 + offset
# full_model = lorentz1 + lorentz2 + offset
full_model = gauss + offset

fit = full_model.fit(data, x=x)

fit.plot()

print(fit.params["iA__fwhm"])
print(fit.params["iA__center"])

In [ ]:
fit.params